<a href="https://colab.research.google.com/github/DanZter/BERT_TRAINING_sentiment_model/blob/master/BERT_TRAINING_sentiment_model_(TPU).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [ ]:
VERSION = "1.5"  #@param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4994  100  4994    0     0  56750      0 --:--:-- --:--:-- --:--:-- 57402
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-1.5 ...
     |████████████████████████████████| 61kB 2.9MB/s 
Uninstalling torch-1.5.1+cu101:
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12
Done updating TPU runtime
  Successfully uninstalled torch-1.5.1+cu101
Uninstalling torchvision-0.6.1+cu101:
  Successfully uninstalled torchvision-0.6.1+cu101
Copying gs://tpu-pytorch/wheels/torch-1.5-cp36-cp36m-linux_x86_64.whl...
- [1 files][ 79.0 MiB/ 79.0 MiB]                                                
Operation completed over 1 objects/79.0 MiB.                                     
Copying gs://tpu-pyto

In [ ]:
# imports pytorch
import torch

# imports the torch_xla package
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.distributed.parallel_loader as pl

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.2.0
Running on TPU  ['10.96.254.26:8470']
INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
!pip install torch
!pip install transformers
!pip install pandas
!pip install -U scikit-learn

     |████████████████████████████████| 778kB 3.3MB/s 
     |████████████████████████████████| 3.0MB 16.7MB/s 
     |████████████████████████████████| 1.1MB 29.9MB/s 
     |████████████████████████████████| 890kB 34.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=81ed45c893b671ec1800b64476274f8b32a677615c3809215146968159a84c56
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 6.9MB 3.2MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
import pandas as pd
import numpy as np
import transformers
from transformers import AdamW, get_linear_schedule_with_warmup
import torch.nn as nn
from sklearn import model_selection
from scipy import stats
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")
# import logging
# logging.basicConfig(level=logging.ERROR)

In [ ]:
MAX_LEN = 256
# TRAIN_BATCH_SIZE = 8
# VALID_BATCH_SIZE = 4
# EPOCHS = 10
# ACCUMULATION = 2
BERT_PATH = "/content/drive/My Drive/Colab Notebooks/input/bert_base_uncased"
TRAINING_FILE = "/content/drive/My Drive/Colab Notebooks/input/IMDB Dataset.csv"
TOKENIZER = transformers.BertTokenizer.from_pretrained(BERT_PATH, do_lower_case=True)
MODEL_PATH ="/content/drive/My Drive/Colab Notebooks/models/bert_sentiment_model.bin"


In [ ]:
class BERTDataset:
    def __init__(self, review, target):
        self.review = review                     # the review text, a list
        self.target = target                     # 0 or 1, a list
        self.tokenizer = TOKENIZER
        self.max_len = MAX_LEN

    def __len__(self):                           # returns the total length of data set
        return len(self.review)

    def __getitem__(self, item):                 # takes an 'item' and returns tokenizer of that item from data set
        review = str(self.review[item])          # converts everything to str incase there exists numbers etc.
        review = " ".join(review.split())        # removes all unnecessary space

        inputs = self.tokenizer.encode_plus(     # encode_plus can encode 2 strings at a time
            review,
            None,                                # since we use only 1 string at a time
            add_special_tokens=True,             # adds cld, sep tokens
            max_length=self.max_len,
            pad_to_max_length=True,
            truncation=True
        )

        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]
        token_type_ids = inputs["token_type_ids"] # since only 1 string token_type_ids are same and unnecessary

        padding_length = self.max_len - len(ids)  # for bert we pad on the right side
        ids = ids + ([0] * padding_length)        # zero times the padding length
        mask = mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'target': torch.tensor(self.target[item], dtype=torch.float)
        }
    """ if we have 2 target outputs then set to torch.long,
    depends on loss function also, from cross-entropy we should use torch.long"""

In [ ]:
def loss_fn(outputs, target):
    return nn.BCEWithLogitsLoss()(outputs, target.view(-1, 1))

def train_fn(data_loader, model, optimizer, device, scheduler=None):
    model.train()

    for bi, d in enumerate(data_loader):
        ids = d["ids"]
        mask = d["mask"]
        token_type_ids = d["token_type_ids"]
        target = d["target"]

        ids = ids.to(device, dtype=torch.long)              # send to TPU device
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        target = target.to(device, dtype=torch.float)

        optimizer.zero_grad()
        outputs = model(
            ids=ids,
            mask=mask,
            token_type_ids=token_type_ids
        )

        loss = loss_fn(outputs, target)        # find loss
        loss.backward()                         # backward propagation

        xm.optimizer_step(optimizer)
        if scheduler is not None:
            scheduler.step()
        if bi % 10 == 0:
            xm.master_print(f"batch_index={bi}, loss={loss}")

        """ stop the optimizer only after a certain number of accumulation steps """


def eval_fn(data_loader, model, device):
    model.eval()
    fin_target = []                         # final targets
    fin_outputs = []                        # final outputs
    with torch.no_grad():
        for bi, d in enumerate(data_loader):
            ids = d["ids"]
            mask = d["mask"]
            token_type_ids = d["token_type_ids"]
            target = d["target"]

            ids = ids.to(device, dtype=torch.long)              # send to cuda device
            mask = mask.to(device, dtype=torch.long)
            token_type_ids = token_type_ids.to(device, dtype=torch.long)
            target = target.to(device, dtype=torch.float)

            outputs = model(
                ids=ids,
                mask=mask,
                token_type_ids=token_type_ids
            )
            # loss = loss_fn(outputs, targets)        # find loss, its bettwer to evaluate loss in eval fn

            fin_target.extend(target.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

    return fin_outputs, fin_target

In [ ]:
class BERTBaseUncased(nn.Module):
    def __init__(self):
        super(BERTBaseUncased, self).__init__()
        self.bert = transformers.BertModel.from_pretrained(BERT_PATH)
        self.bert_drop = nn.Dropout(0.3)
        self.out = nn.Linear(768, 1)
        """ 768: bert we use have 768 features | 1: binary classification
        if we use 2, we need to change the loss function"""

    def forward(self, ids, mask, token_type_ids):
        _, o2 = self.bert(
            ids,
            attention_mask=mask,
            token_type_ids=token_type_ids
        )
        """ We have 2 outputs from a BERT model
         o1(last hidden state): is the sequence of hidden states. eg. if we have 512 tokens (MAX_LEN), 
         we have 512 vectors of size 768 for each batch. We can use out1 to max pooling or averge pooling
         o2(pooler output from bert pooler layer): we get vector of size 768 for each sample in batch"""
        bo = self.bert_drop(o2)                                 # drop-out
        output = self.out(bo)                                   # linear-layer
        return output

In [ ]:
from sklearn import metrics

def run(index, flags):

    flags['MAX_LEN'] = 256
    flags['TRAIN_BATCH_SIZE'] = 32  # 9.47min for BS=8
                                    # 5.54min for BS=32
    flags['VALID_BATCH_SIZE'] = 16
    flags['EPOCHS'] = 10
    flags['seed'] = 1234  
    torch.manual_seed(flags['seed'])

    dfx = pd.read_csv(TRAINING_FILE).fillna("none")
    dfx.sentiment = dfx.sentiment.apply(  # can use label encoding
        lambda x: 1 if x == "positive" else 0  # can use map fn
    )

    df_train, df_valid = model_selection.train_test_split(
        dfx,
        test_size=0.1,
        random_state=42,
        stratify=dfx.sentiment.values  # when split both train and val have same positive to negative sample ratio
    )

    df_train = df_train.reset_index(drop=True)  # 0 to length of df_train
    df_valid = df_valid.reset_index(drop=True)  # 0 to length of df_valid

    train_dataset = BERTDataset(
        review=df_train.review.values,
        target=df_train.sentiment.values
    )
    train_sampler = torch.utils.data.DistributedSampler(
       train_dataset,
       num_replicas=xm.xrt_world_size(),
       rank=xm.get_ordinal(),
       shuffle=True 
    )

    train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=flags['TRAIN_BATCH_SIZE'],
        sampler=train_sampler
    )
####################################################
    valid_dataset = BERTDataset(
        review=df_valid.review.values,
        target=df_valid.sentiment.values
    )
    valid_sampler = torch.utils.data.DistributedSampler(
       valid_dataset,
       num_replicas=xm.xrt_world_size(),
       rank=xm.get_ordinal()
    )
    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=flags['VALID_BATCH_SIZE'],
        sampler=valid_sampler
    )

    # device = "cuda"                   # cuda
    device = xm.xla_device()            # tpu

    model = BERTBaseUncased().to(device)
    # lr = 3e-5 *xm.xrt_world_size()
    param_optimizer = list(model.named_parameters())  # specify parameters to train
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    """ These parameters are adjustable, we should take a look at different layers and
    the decay we want, how much learning rate etc."""

    num_train_steps = int(len(df_train) / flags['TRAIN_BATCH_SIZE'] / xm.xrt_world_size() * flags['EPOCHS'])
    optimizer = AdamW(optimizer_parameters, lr=3e-5)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=num_train_steps
    )

    # model = nn.DataParallel(model)              # converting to multi gpu model

    best_accuracy = 0
    for epoch in tqdm(range(flags['EPOCHS']), total=flags['EPOCHS']):

        para_loader = pl.ParallelLoader(train_data_loader, [device])
        train_fn(para_loader.per_device_loader(device), model, optimizer, device, scheduler)

        para_loader = pl.ParallelLoader(valid_data_loader, [device])
        outputs, target = eval_fn(para_loader.per_device_loader(device), model, device)
        outputs = np.array(outputs) >= 0.5
        accuracy = metrics.accuracy_score(target, outputs)
        xm.master_print(f"Accuracy score = {accuracy}")
        xm.save(model.state_dict(), MODEL_PATH)          # tpu  # saving the model only if it improves


flags = {}
xmp.spawn(run, args=(flags,), nprocs=8, start_method='fork')

  0%|          | 0/10 [00:00<?, ?it/s]

batch_index=0, loss=0.712925910949707
batch_index=10, loss=0.572005033493042
batch_index=20, loss=0.41632986068725586
batch_index=30, loss=0.3658309280872345
batch_index=40, loss=0.3177637457847595
batch_index=50, loss=0.3634778559207916
batch_index=60, loss=0.2803698182106018
batch_index=70, loss=0.10925929993391037
batch_index=80, loss=0.3809845447540283
batch_index=90, loss=0.2588035464286804
batch_index=100, loss=0.2224305421113968
batch_index=110, loss=0.14605173468589783
batch_index=120, loss=0.2638278007507324
batch_index=130, loss=0.1329445242881775
batch_index=140, loss=0.14641815423965454
batch_index=150, loss=0.3915320038795471
batch_index=160, loss=0.21254122257232666
batch_index=170, loss=0.25655731558799744
Accuracy score = 0.9088


 10%|█         | 1/10 [04:55<44:15, 295.10s/it]

batch_index=0, loss=0.140375554561615
batch_index=10, loss=0.0999813824892044
batch_index=20, loss=0.1271202713251114
batch_index=30, loss=0.19300946593284607
batch_index=40, loss=0.08767985552549362
batch_index=50, loss=0.3392219841480255
batch_index=60, loss=0.08976869285106659
batch_index=70, loss=0.1573590338230133
batch_index=80, loss=0.16194798052310944
batch_index=90, loss=0.0737970694899559
batch_index=100, loss=0.09647437930107117
batch_index=110, loss=0.024589549750089645
batch_index=120, loss=0.07751443982124329
batch_index=130, loss=0.03608041629195213
batch_index=140, loss=0.02505209855735302
batch_index=150, loss=0.13865438103675842
batch_index=160, loss=0.09108159691095352
batch_index=170, loss=0.0880570262670517
Accuracy score = 0.9088


 20%|██        | 2/10 [09:52<39:26, 295.81s/it]

...

 80%|████████  | 8/10 [39:33<09:53, 296.66s/it]

batch_index=0, loss=0.000792522681877017
batch_index=10, loss=0.0007040736963972449
batch_index=20, loss=0.0007748660864308476
batch_index=30, loss=0.0008955808007158339
batch_index=40, loss=0.0009666133555583656
batch_index=50, loss=0.10031946003437042
batch_index=60, loss=0.0006993578281253576
batch_index=70, loss=0.000798467721324414
batch_index=80, loss=0.0012200031196698546
batch_index=90, loss=0.001264916267246008
batch_index=100, loss=0.001317988382652402
batch_index=110, loss=0.000631515693385154
batch_index=120, loss=0.0009193432633765042
batch_index=130, loss=0.0007406898657791317
batch_index=140, loss=0.0007438664324581623
batch_index=150, loss=0.0009169376571662724
batch_index=160, loss=0.0014492084737867117
batch_index=170, loss=0.0011345382081344724
Accuracy score = 0.9152


 90%|█████████ | 9/10 [44:32<04:57, 297.27s/it]

batch_index=0, loss=0.0006689302390441298
batch_index=10, loss=0.0006125119398348033
batch_index=20, loss=0.0006922961329109967
batch_index=30, loss=0.0007985163829289377
batch_index=40, loss=0.00079924869351089
batch_index=50, loss=0.09079866856336594
batch_index=60, loss=0.0006292837206274271
batch_index=70, loss=0.0007417433080263436
batch_index=80, loss=0.0010458879405632615
batch_index=90, loss=0.001201584585942328
batch_index=100, loss=0.0011632515816017985
batch_index=110, loss=0.0005571283982135355
batch_index=120, loss=0.0008400609949603677
batch_index=130, loss=0.0006612197612412274
batch_index=140, loss=0.0006405180902220309
batch_index=150, loss=0.0007852810667827725
batch_index=160, loss=0.0012958534061908722
batch_index=170, loss=0.0010764047037810087
Accuracy score = 0.9152


100%|██████████| 10/10 [50:18<00:00, 310.72s/it]



100%|██████████| 10/10 [50:13<00:00, 301.36s/it]

100%|██████████| 10/10 [50:13<00:00, 301.39s/it]
